# Backtesting Fibonacci Retracement Post Wave 1 Peak

In [1]:
import requests
import sys
import json
import base64
import time
import logging
from solders.keypair import Keypair
from solders.transaction import VersionedTransaction 
from solana.rpc.api import Client
from solana.rpc.types import TxOpts
from pprint import pprint
from functools import lru_cache
from datetime import datetime, timedelta
import pytz
import json
import pandas as pd
from pprint import pprint 
from IPython.display import display, HTML
import time
import os

# Path to Master Functions and Keys
current_dir = os.getcwd()
root_dir = os.path.abspath(os.path.join(current_dir, '..', '..', '..', '..'))
sys.path.append(root_dir)

import Birdeye.Basics.Basics_Mastered as birdeye_functions

{"success":true,"data":["solana","ethereum","arbitrum","avalanche","bsc","optimism","polygon","base","zksync"]}


# Wallet and API Key

In [2]:
API_Key = birdeye_functions.API_Key
wallet = birdeye_functions.wallet
chain = birdeye_functions.chain # Choose between: solana,ethereum,arbitrum,avalanche,bsc,optimism,polygon,base,zksync


# Filter Tokens by Market Cap Range

#### Established Tokens

In [4]:
sort_by = "mc"  # mc, v24hUSD, v24hChangePercent 
sort_type = "desc"  # asc, desc
min_liquidity = 100000  # Minimum liquidity in USD (100k)
min_volume_24h = 1000000  # Minimum 24-hour trading volume in USD
min_market_cap = 1000000  # Minimum market cap in USD (1 million)
max_market_cap = 900000000  # Maximum market cap in USD (900 million)
total_tokens = 300  # Change this to the number of tokens you want to retrieve

# Get the list of established tokens
established_tokens_filtered = birdeye_functions.get_token_list(sort_by, sort_type, min_liquidity, min_volume_24h, min_market_cap, max_market_cap, total_tokens, chain, API_Key)
#print(established_tokens_filtered)

# Set the index to 'Address'
established_tokens_filtered = established_tokens_filtered.set_index('Address')

# Get the list of addresses
established_tokens_addresses = established_tokens_filtered.index.tolist()

# Get Token Overview Data for established tokens
established_tokens_data = birdeye_functions.get_token_trade_data_multi(established_tokens_addresses, API_Key)

# Get a list of all unique attributes across all DataFrames
all_attributes = set()
for token, df in established_tokens_data.items():
    if isinstance(df, pd.DataFrame) and not df.empty:
        all_attributes.update(df['Attribute'].unique())

# Convert the set to a sorted list
columns_array = sorted(list(all_attributes))

# Create the master DataFrame
established_tokens_master_overview = pd.DataFrame(index=established_tokens_data.keys(), columns=columns_array)

# Fill the DataFrame with values
for token, df in established_tokens_data.items():
    if isinstance(df, pd.DataFrame) and not df.empty:
        for _, row in df.iterrows():
            established_tokens_master_overview.at[token, row['Attribute']] = row['Value']

# Set the index name to 'Address'
established_tokens_master_overview.index.name = 'Address'

# Optional: Save to CSV
established_tokens_master_overview.to_csv('Data/established_tokens_master_overview.csv')


                            Name    Symbol    Market Cap    Volume 24h  \
0     Marinade staked SOL (mSOL)      mSOL  8.329887e+08  1.306101e+07   
1            cat in a dogs world       MEW  7.628188e+08  4.266492e+06   
2               Jupiter Perps LP       JLP  7.553799e+08  1.866031e+07   
3                           Kima      KIMA  6.951236e+08  9.559111e+07   
4                   BOOK OF MEME      BOME  6.488343e+08  4.187847e+06   
5             Jupiter Staked SOL    JupSOL  5.799024e+08  1.247586e+07   
6                       GIGACHAD      GIGA  4.798700e+08  4.723867e+06   
7                       deBr1dge       DBR  4.094293e+08  1.552216e+08   
8               Goatseus Maximus      GOAT  3.838756e+08  1.722467e+08   
9                       deBridge       DBR  3.625051e+08  2.815902e+06   
10                         GRA$$     GRA$$  2.990588e+08  3.464609e+08   
11  BlazeStake Staked SOL (bSOL)      bSOL  2.799370e+08  7.607228e+06   
12           Rebe1 Bots Ep1c War      

#### New Token Listings

In [10]:
# Example usage:
days_back = 1  # Change this value to look back fewer days (1 for 1 day, 2 for 2 days, 3 for 3 days max)
hours_back = 0  # Change this value to look back fewer hours within the selected day
                # Note: when hours_back is 1, it looks back for all new tokens within 1 hour.
                # when hours_back is 0, it looks back for all new tokens within the selected minutes.
minutes_back = 5  # Change this value to look back fewer minutes within the selected hour
liquidity_filter = 10000

new_tokens = birdeye_functions.get_new_listings(days_back, hours_back, minutes_back, API_Key, liquidity_filter)

#new_tokens_columns = new_tokens[0].columns.tolist()
#print(new_tokens_columns)

new_tokens_filtered = new_tokens[1].rename(columns={
    'address': 'Address'
})

# Get List of New Tokens Addresses
new_tokens_address = new_tokens_filtered['Address'].tolist()

# Set Index
new_tokens_filtered = new_tokens_filtered.set_index('Address')
new_tokens_filtered.head(10)

,symbol,name,decimals,liquidityAddedAt,logoURI,liquidity
Address,,,,,,
CLdcRYuGZXh26eQ1iujwnN2yLBuvne7MT1JbFxgcpump,FIG,Instagram's Smallest Cat,6,2024-10-20 05:14:31,https://ipfs.io/ipfs/QmTK8vhF5EEX7hYjgDvMFqCJM...,25958.213831
qaiqggVdicmx6zUcSwGkw6Yz4gAetxQArCFagqypump,XOXO,EGIRL,6,2024-10-20 05:10:57,https://ipfs.io/ipfs/QmWYAV773Tw2baEctT2i15GLj...,32092.467289
GpnKqJBG1Evs8j1f1qj1T6pxrbJSGNuj4TUUp1vGLiRA,BOOM,Book of MAGA,6,2024-10-20 05:07:10,https://gateway.pinata.cloud/ipfs/QmaYbmkNT6Gt...,33204.499224


In [11]:
# Example usage:
days_back = 1  # Change this value to look back fewer days (1 for 1 day, 2 for 2 days, 3 for 3 days max)
hours_back = 0  # Change this value to look back fewer hours within the selected day
                # Note: when hours_back is 1, it looks back for all new tokens within 1 hour.
                # when hours_back is 0, it looks back for all new tokens within the selected minutes.
minutes_back = 5  # Change this value to look back fewer minutes within the selected hour
liquidity_filter = 10000

new_tokens = birdeye_functions.get_new_listings(days_back, hours_back, minutes_back, API_Key, liquidity_filter)

#new_tokens_columns = new_tokens[0].columns.tolist()
#print(new_tokens_columns)

new_tokens_filtered = new_tokens[1].rename(columns={
    'address': 'Address'
})

# Get List of New Tokens Addresses
new_tokens_address = new_tokens_filtered['Address'].tolist()

# Set Index
new_tokens_filtered = new_tokens_filtered.set_index('Address')

# Get Token Overview Data

# Get Token Overview Data
token_overview_data = birdeye_functions.get_token_trade_data_multi(new_tokens_address, API_Key)

# Get a list of all unique attributes across all DataFrames
all_attributes = set()
for token, df in token_overview_data.items():
    if isinstance(df, pd.DataFrame) and not df.empty:
        all_attributes.update(df['Attribute'].unique())

# Convert the set to a sorted list
columns_array = sorted(list(all_attributes))

#print("List of columns in an array:")
#print(columns_array)

# Create the master DataFrame
new_tokens_master_overview = pd.DataFrame(index=token_overview_data.keys(), columns=columns_array)

# Fill the DataFrame with values
for token, df in token_overview_data.items():
    if isinstance(df, pd.DataFrame) and not df.empty:
        for _, row in df.iterrows():
            new_tokens_master_overview.at[token, row['Attribute']] = row['Value']

# Set the index name to 'Address'
new_tokens_master_overview.index.name = 'Address'

# Filter out rows where 'Error' column is not NaN
new_tokens_master_overview = new_tokens_master_overview[new_tokens_master_overview['Error'].isna()]
new_tokens_master_overview["liquidityAddedAt"] = new_tokens_filtered["liquidityAddedAt"]
new_tokens_master_overview.columns

new_tokens_master_overview.head(20)

# Optional: Save to CSV
# new_tokens_master_overview.to_csv('Data/new_tokens_master_overview.csv')

KeyError: 'Error'

In [72]:
address = "8zyqBWF2rebKAnfxpAoAVDdLex3e8Ny1G15ahpb6VxYn"

url = f"https://public-api.birdeye.so/defi/v3/token/market-data?address={address}"

headers = {
    "accept": "application/json",
    "X-API-KEY": API_Key
}

response = requests.get(url, headers=headers)

print(response.text)

{"data":{"address":"8zyqBWF2rebKAnfxpAoAVDdLex3e8Ny1G15ahpb6VxYn","price":0.00010590472973542112,"liquidity":475.90238445424035,"supply":21000000,"marketcap":2223.9993244438433,"circulating_supply":21000000,"circulating_marketcap":2223.9993244438433},"success":true}


# Get Token List

#### Established Tokens

#### New Token Listings

In [33]:
def get_new_listings(days_back, hours_back, minutes_back, API_Key, liquidity_filter):
    """
    Fetch new token listings data.

    Args:
    days_back (int): Number of days to look back (1, 2, or 3).
    hours_back (int): Number of hours to look back within the selected day.
    minutes_back (int): Number of minutes to look back within the selected hour.
    API_Key (str): Your Birdeye API key.
    liquidity_filter (float): Minimum liquidity for filtered results.

    Returns:
    tuple: A tuple containing two DataFrames (full_df, filtered_df) with new listings data.
    """
    def iso8601_to_timestamp(iso_string):
        return int(datetime.fromisoformat(iso_string.replace('Z', '+00:00')).timestamp())

    end_time = datetime.now(pytz.UTC)
    
    if days_back == 1:
        start_time = end_time - timedelta(hours=hours_back, minutes=minutes_back)
    elif days_back == 2:
        start_time = end_time - timedelta(days=1, hours=hours_back, minutes=minutes_back)
    elif days_back == 3:
        start_time = end_time - timedelta(days=2, hours=hours_back, minutes=minutes_back)
    else:
        raise ValueError("days_back must be 1, 2, or 3")
    
    all_tokens = []
    current_time = int(end_time.timestamp())
    
    while current_time > int(start_time.timestamp()):
        url = f"https://public-api.birdeye.so/defi/v2/tokens/new_listing?time_to={current_time}&limit=10"

        headers = {
            "accept": "application/json",
            "X-API-KEY": API_Key
        }

        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            data = response.json()
            if 'data' in data and 'items' in data['data']:
                new_tokens = data['data']['items']
                if not new_tokens:
                    break
                all_tokens.extend(new_tokens)
                current_time = min(iso8601_to_timestamp(token['liquidityAddedAt']) for token in new_tokens)
            else:
                print(f"Unexpected response structure")
                print("Response:", response.text)
                break
        else:
            print(f"Request failed with status code: {response.status_code}")
            print("Response:", response.text)
            break

        # Respect rate limits
        time.sleep(1)

    # Remove duplicates based on 'address'
    unique_tokens = list({token['address']: token for token in all_tokens}.values())
    
    if unique_tokens:
        # Create full DataFrame
        df_full = pd.DataFrame(unique_tokens)
        
        # Convert liquidityAddedAt to datetime
        df_full['liquidityAddedAt'] = pd.to_datetime(df_full['liquidityAddedAt'])
        
        # Sort by liquidityAddedAt in descending order
        df_full = df_full.sort_values('liquidityAddedAt', ascending=False)
        
        # Apply filters for the second table
        df_filtered = df_full[
            (df_full['name'].notna() & (df_full['name'] != 'None')) & 
            (df_full['logoURI'].notna() & (df_full['logoURI'] != 'None')) & 
            (df_full['liquidity'].astype(float) >= liquidity_filter)
        ]
        
        return df_full, df_filtered
    else:
        return pd.DataFrame(), pd.DataFrame()

# Example usage:
days_back = 1  # Change this value to look back fewer days (1 for 1 day, 2 for 2 days, 3 for 3 days max)
hours_back = 0  # Change this value to look back fewer hours within the selected day
                # Note: when hours_back is 1, it looks back for all new tokens within 1 hour.
                # when hours_back is 0, it looks back for all new tokens within the selected minutes.
minutes_back = 5  # Change this value to look back fewer minutes within the selected hour
liquidity_filter = 10000
new_tokens_filtered = get_new_listings(days_back, hours_back, minutes_back, API_Key, liquidity_filter)
#print(df_filtered)
print(new_tokens_filtered[0].columns.tolist())


['address', 'symbol', 'name', 'decimals', 'liquidityAddedAt', 'logoURI', 'liquidity']


# Get Token OHLCV and Save

In [30]:
def get_ohlcv_data_multi(tokens, API_Key, timeframes=None):
    """
    Fetch OHLCV data for multiple tokens and specified timeframes.

    Args:
    tokens (list): List of token addresses to fetch data for.
    API_Key (str): Your Birdeye API key.
    timeframes (list): List of timeframes to fetch data for. Default is ['15m'].

    Returns:
    dict: A nested dictionary with tokens and timeframes as keys and DataFrames as values.
    """
    if timeframes is None:
        timeframes = ['15m']

    timeframes_data = {
        '1m': 1 * 24 * 60,    # 1 day of 1-minute data
        '3m': 3 * 24 * 20,    # 3 days of 3-minute data
        '5m': 7 * 24 * 12,    # 7 days of 5-minute data
        '15m': 25 * 24 * 4,   # 25 days of 15-minute data
        '30m': 50 * 24 * 2,   # 50 days of 30-minute data
        '1H': 100 * 24,       # 100 days of 1-hour data
        '2H': 200 * 12,       # 200 days of 2-hour data
        '4H': 400 * 6,        # 400 days of 4-hour data
        '6H': 600 * 4,        # 600 days of 6-hour data
        '8H': 800 * 3,        # 800 days of 8-hour data
        '12H': 1200 * 2,      # 1200 days of 12-hour data
        '1D': 5 * 365         # 5 years of daily data
    }

    results = {}

    for token in tokens:
        results[token] = {}
        for timeframe in timeframes:
            if timeframe not in timeframes_data:
                print(f"Invalid timeframe: {timeframe}. Skipping.")
                continue

            # Calculate start and end times
            end_time = datetime.now(pytz.UTC)
            start_time = end_time - timedelta(hours=timeframes_data[timeframe])

            # Convert to Unix timestamps
            time_from = int(start_time.timestamp())
            time_to = int(end_time.timestamp())

            url = f"https://public-api.birdeye.so/defi/ohlcv?address={token}&type={timeframe}&time_from={time_from}&time_to={time_to}"

            headers = {
                "accept": "application/json",
                "X-API-KEY": API_Key
            }

            response = requests.get(url, headers=headers)

            if response.status_code == 200:
                try:
                    data = json.loads(response.text)
                    
                    if 'data' in data and 'items' in data['data']:
                        items = data['data']['items']
                        
                        if items:
                            df = pd.DataFrame(items)
                            df['datetime'] = pd.to_datetime(df['unixTime'], unit='s')
                            df.set_index('datetime', inplace=True)
                            columns_order = [col for col in df.columns if col != 'unixTime']
                            df = df[columns_order]

                            results[token][timeframe] = df
                            
                            # Print the head of each token-timeframe table
                            print(f"\nHead of {token} - {timeframe} table:")
                            print(df.head())
                        else:
                            results[token][timeframe] = pd.DataFrame()
                            print(f"\nNo data items found for token {token}, timeframe {timeframe}.")
                    else:
                        results[token][timeframe] = pd.DataFrame()
                        print(f"\nUnexpected response structure for token {token}, timeframe {timeframe}.")
                except json.JSONDecodeError:
                    results[token][timeframe] = pd.DataFrame()
                    print(f"\nFailed to parse JSON response for token {token}, timeframe {timeframe}.")
            else:
                results[token][timeframe] = pd.DataFrame()
                print(f"\nRequest failed with status code: {response.status_code} for token {token}, timeframe {timeframe}.")

    return results


In [31]:
# Example usage:
tokens = filtered_token_list
result = get_ohlcv_data_multi(tokens, API_Key, timeframes=['1m']) # ['1m', '3m', '5m', '15m', '30m', '1H', '2H', '4H', '6H', '8H', '12H', '1D']


Head of mSoLzYCxHdYgdzU16g5QSh3i5K3z3KZK7ytfqcJm7So - 1m table:
                                                         address           c  \
datetime                                                                       
2024-08-20 17:13:00  mSoLzYCxHdYgdzU16g5QSh3i5K3z3KZK7ytfqcJm7So  173.034117   
2024-08-20 17:14:00  mSoLzYCxHdYgdzU16g5QSh3i5K3z3KZK7ytfqcJm7So  173.008842   
2024-08-20 17:15:00  mSoLzYCxHdYgdzU16g5QSh3i5K3z3KZK7ytfqcJm7So  173.014879   
2024-08-20 17:16:00  mSoLzYCxHdYgdzU16g5QSh3i5K3z3KZK7ytfqcJm7So  173.091518   
2024-08-20 17:17:00  mSoLzYCxHdYgdzU16g5QSh3i5K3z3KZK7ytfqcJm7So  173.122128   

                              h           l           o type          v  
datetime                                                                 
2024-08-20 17:13:00  173.120417  173.034117  173.077397   1m   6.944023  
2024-08-20 17:14:00  173.034117  173.008842  173.034117   1m  12.035376  
2024-08-20 17:15:00  173.078423  172.988920  173.008842   1m  91.484238  
2024

# Implement Fibonacci Retracement Optimization